![](../pictures/Databand-logo.png)

# Deploying Databand

# 1. Preparing the environment

## 1.1 Logging to the cluster

First of all, we need a way to login to the custer from the command line. Open the Openshift Console and proceed as follows:



![](../pictures/login1.png)



After clicking on `Copy login command` a new tab displays the contents of the command you need to copy to the clipboard 



![](../pictures/login2.png)


You need to paste the contents of the clipboard into the next cell

In [1]:
# Replace the command with your own one inside the single quotes and run the cell
# Example OC_LOGIN_COMMAND='oc login --token=sha256~3bR5KXgwiUoaQiph2_kIXCDQnVfm_HQy3YwU2m-UOrs --server=https://c109-e.us-east.containers.cloud.ibm.com:31656'
OC_LOGIN_COMMAND='_replace_this_string_by_pasting_the_clipboard_'

Now, you can test that it works by executing the next cell:

In [2]:
echo $OC_LOGIN_COMMAND
$OC_LOGIN_COMMAND

oc login --token=sha256~YUoJrPZBkAtjead6CDeSveXNplatuHwKqId5xXmqRVM --server=https://c102-e.us-east.containers.cloud.ibm.com:31723
Logged into "https://c102-e.us-east.containers.cloud.ibm.com:31723" as "IAM#angelito@de.ibm.com" using the token provided.

You have access to 67 projects, the list has been suppressed. You can list all projects with 'oc projects'

Using project "default".


Please remember this command as you may need to issue it again if, for example,  you leave the jupyter session.

**Warning:** The token will expire and changes after some time. Don't be surprised if you need to repeat the same thing tomorrow for logging into the cluster again

## 1.2 Expose the registry

We will need to upload some docker images to the cluster image registry but, by default, it is not accesible after the provisioning. So, we will expose it with this command:

In [3]:
# This command exposes the registry
oc patch configs.imageregistry.operator.openshift.io/cluster --patch '{"spec":{"defaultRoute":true}}' --type=merge

config.imageregistry.operator.openshift.io/cluster patched


To verify if the registry is really exposed now, run the following command:

In [4]:
# This command check if there is a network route associated the the image registry
oc get route -n openshift-image-registry 

NAME            HOST/PORT                                                                                                                                   PATH   SERVICES         PORT    TERMINATION   WILDCARD
default-route   default-route-openshift-image-registry.itzroks-1100005cc8-iw6gvy-6ccd7f378ae819553d37d5f2ee142bd6-0000.us-east.containers.appdomain.cloud          image-registry   <all>   reencrypt     None


If something like "No resources found" is displayed, the patch command did not work. Stop here and fix it before continuing.


Now, we need to select the actual route out of this complicated output. Just run this:
<a id='registry_url'></a>

In [5]:

# This command extracts the portion of the registry route
# Example: default-route-openshift-image-registry.itzroks-1100005cc8-4b30y2-4b4a324f027aea19c5cbc0c3275c4656-0000.us-east.containers.appdomain.cloud
REGISTRY_URL=$(oc get route -n openshift-image-registry | grep default-route | awk '{print $2}')
echo $REGISTRY_URL

default-route-openshift-image-registry.itzroks-1100005cc8-iw6gvy-6ccd7f378ae819553d37d5f2ee142bd6-0000.us-east.containers.appdomain.cloud


## 2. Preparing Docker images 

### 2.1 Loading images locally

In the last part of the of the [prerequisites section](./0_prerequisites.ipynb) we downloaded the file `databand-1.0.19-helm-chart.tar.gz` (960MB). After decompressing it, we see something like this:

    (base) Angel@AngelMac databand-1.0.19-helm-chart % ls -l
    total 5050480
    -rw-r--r--@ 1 Angel  staff      318291 13 feb 14:07 databand-1.0.19-7.tgz
    -rw-------@ 1 Angel  staff  2579398656 13 feb 14:09 databand-v1.0.19.7-images.tar`

Our task now is importing the databand images in the reqistry of our local computer. First, let's define the directory where these two files are located after decompressing the databand package file:

In [6]:
# Copy the directory where you decompressed the databand package into the clipboard and paste it in this cell. Please observe the single quotes.
# Example: DATABAND_UNCOMPRESSED_DIR='/Users/Angel/Downloads/databand-1.0.19-helm-chart'
# Then, run this cell:
DATABAND_UNCOMPRESSED_DIR='_paste_here_the_clipboard_'
echo $DATABAND_UNCOMPRESSED_DIR

/Users/Angel/Downloads/databand-1.0.19-helm-chart


In [ ]:
# Run this cell. It will produce a very long output
cd $DATABAND_UNCOMPRESSED_DIR
ls -l
DATABAND_IMAGES=$(ls databand*images.tar) 
docker load -i $DATABAND_IMAGES

Run this command to verify that the load went well:

In [9]:
docker images

REPOSITORY                                                             TAG                    IMAGE ID       CREATED         SIZE
registry.gitlab.com/databand/databand-deploy/dbnd-datasource-monitor   python-v1.0.19.7       5dcb080260b4   2 weeks ago     457MB
registry.gitlab.com/databand/databand-deploy/dbnd-ml-trainer           python-v1.0.19.7       3376fd4ed290   2 weeks ago     619MB
registry.gitlab.com/databand/databand-deploy/dbnd-webserver            python-v1.0.19.7       84f9441ea6d6   2 weeks ago     619MB
registry.gitlab.com/databand/databand-deploy/dbnd-datastage-monitor    python-v1.0.19.7       c395af0c2cd9   2 weeks ago     225MB
registry.gitlab.com/databand/databand-deploy/dbnd-dbt-monitor          python-v1.0.19.7       019bed143ae0   2 weeks ago     224MB
registry.gitlab.com/databand/databand-deploy/dbnd-alert-defs-syncer    python-v1.0.19.7       a1cb3ed34b50   2 weeks ago     325MB
registry.gitlab.com/databand/databand-deploy/dbnd-webapp               nginx-v1.0.19

### 2.2. Tagging images

Now, we need to issue very cumbersome commands. Pay attention to the following steps.

First, we need to ensure that we know the route to the registry. We set the variable `REGISTRY_URL` in a [previous cell](#registry_url) where we extracted just the route of the registry. We can run it once again to be sure that it is set and remember the manual circumvention as explained [here](#manual_registry_url) in case that the output is different from the expected.

In [10]:
# This command extracts the portion of the registry route. We did it before but you can run it again to be sure
# Example of expected output: default-route-openshift-image-registry.itzroks-1100005cc8-4b30y2-4b4a324f027aea19c5cbc0c3275c4656-0000.us-east.containers.appdomain.cloud
REGISTRY_URL=$(oc get route -n openshift-image-registry | grep default-route | awk '{print $2}')
echo $REGISTRY_URL

default-route-openshift-image-registry.itzroks-1100005cc8-iw6gvy-6ccd7f378ae819553d37d5f2ee142bd6-0000.us-east.containers.appdomain.cloud


Then, we need to retrieve our actual userid and password in the OpenShift cluster and log into the registry

In [11]:
# Run this cell. It is necessary to login to the registry (apart from being logged into the cluster itself)
REGISTRY_USER=$(oc whoami)
echo $REGISTRY_USER
REGISTRY_PASS=$(oc whoami -t)
echo $REGISTRY_PASS

docker login -u $REGISTRY_USER -p $REGISTRY_PASS $REGISTRY_URL


IAM#angelito@de.ibm.com
sha256~YUoJrPZBkAtjead6CDeSveXNplatuHwKqId5xXmqRVM
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Login Succeeded


Now, we create a project in the cluster. Name it `databand` just for simplicity

In [12]:
# Run this cell. It will create an OpenShift project, which is equivalent to a namespace in Kubernetes terminology
DATABAND_PROJECT=databand 
oc new-project $DATABAND_PROJECT

Now using project "databand" on server "https://c102-e.us-east.containers.cloud.ibm.com:31723".

You can add applications to this project with the 'new-app' command. For example, try:

    oc new-app rails-postgresql-example

to build a new example application in Ruby. Or use kubectl to deploy a simple Kubernetes application:

    kubectl create deployment hello-node --image=k8s.gcr.io/e2e-test-images/agnhost:2.33 -- /agnhost serve-hostname



Finally, we have everything to tag our container images. Please review the output of this command:

In [13]:
# Run this command to check that we will produce correct tags
# it only echoes the commands but does not issue them
docker images | grep 'dbnd-' | while read a b c
do
   my_image=$(echo ${a}:${b} | awk -F"/" '{print $NF}')
  
   echo docker tag ${a}:${b} $REGISTRY_URL/$DATABAND_PROJECT/$my_image
done

docker tag registry.gitlab.com/databand/databand-deploy/dbnd-datasource-monitor:python-v1.0.19.7 default-route-openshift-image-registry.itzroks-1100005cc8-iw6gvy-6ccd7f378ae819553d37d5f2ee142bd6-0000.us-east.containers.appdomain.cloud/databand/dbnd-datasource-monitor:python-v1.0.19.7
docker tag registry.gitlab.com/databand/databand-deploy/dbnd-ml-trainer:python-v1.0.19.7 default-route-openshift-image-registry.itzroks-1100005cc8-iw6gvy-6ccd7f378ae819553d37d5f2ee142bd6-0000.us-east.containers.appdomain.cloud/databand/dbnd-ml-trainer:python-v1.0.19.7
docker tag registry.gitlab.com/databand/databand-deploy/dbnd-webserver:python-v1.0.19.7 default-route-openshift-image-registry.itzroks-1100005cc8-iw6gvy-6ccd7f378ae819553d37d5f2ee142bd6-0000.us-east.containers.appdomain.cloud/databand/dbnd-webserver:python-v1.0.19.7
docker tag registry.gitlab.com/databand/databand-deploy/dbnd-datastage-monitor:python-v1.0.19.7 default-route-openshift-image-registry.itzroks-1100005cc8-iw6gvy-6ccd7f378ae819553d

If you see a series of commands with this format...

    docker tag _image-in-local-registry_  _registry-route_/databand/_image_name_ 

...you can then run safely this cell:

In [14]:
# Run this cell to tag the databand images
docker images | grep 'dbnd-' | while read a b c
do
   my_image=$(echo ${a}:${b} | awk -F"/" '{print $NF}')
  
   docker tag ${a}:${b} $REGISTRY_URL/$DATABAND_PROJECT/$my_image
done

### 2.3 Pushing images

If the last command worked fine, run this one as well but now be more patient. It will upload the local images to the OpenShift cluster

In [ ]:
# Run this cell to push the databand images. Expect a long output
docker images | grep 'dbnd-' | while read a b c
do
   my_image=$(echo ${a}:${b} | awk -F"/" '{print $NF}')
  
   docker push $REGISTRY_URL/$DATABAND_PROJECT/$my_image
done

## 3. Helm Chart deployment 


We unpacked the databand media package in a previous step to load the container images (section 2.1). Let's go to that working directory again:

In [16]:
# Copy the directory where you decompressed the databand package into the clipboard and paste it in this cell. Please observe the single quotes.
# Example: DATABAND_UNCOMPRESSED_DIR='/Users/Angel/Downloads/databand-1.0.19-helm-chart'
# Then, run this cell:
DATABAND_UNCOMPRESSED_DIR='_paste_here_the_clipboard_'
echo $DATABAND_UNCOMPRESSED_DIR

/Users/Angel/Downloads/databand-1.0.19-helm-chart


You can verify that there is a big file with the docker images and a smaller one with the helm charts

In [17]:
cd $DATABAND_UNCOMPRESSED_DIR
pwd
ls -l

/Users/Angel/Downloads/databand-1.0.19-helm-chart
total 5038512
-rw-r--r--@ 1 Angel  staff      318291 Feb 13 14:07 databand-1.0.19-7.tgz
-rw-------@ 1 Angel  staff  2579398656 Feb 13 14:09 databand-v1.0.19.7-images.tar


Now, lets unpack the helm charts and we will see that the new subdirectory `databand`

In [18]:
tar -zxf databand-1.0.19-7.tgz
ls -l


total 5038512
drwxr-xr-x  15 Angel  staff         480 Mar  2 13:53 databand
-rw-r--r--@  1 Angel  staff      318291 Feb 13 14:07 databand-1.0.19-7.tgz
-rw-------@  1 Angel  staff  2579398656 Feb 13 14:09 databand-v1.0.19.7-images.tar


Go to this directory and have a look at the list of files.

In [19]:
cd databand
pwd
ls -l

/Users/Angel/Downloads/databand-1.0.19-helm-chart/databand
total 184
-rw-r--r--@  1 Angel  staff   1098 Feb 13 14:07 Chart.lock
-rw-r--r--@  1 Angel  staff   1624 Feb 13 14:07 Chart.yaml
-rw-r--r--@  1 Angel  staff  11343 Feb 13 14:07 LICENSE
drwxr-xr-x   3 Angel  staff     96 Mar  2 13:53 LICENSES
-rw-r--r--@  1 Angel  staff  18137 Feb 13 14:07 README.md
drwxr-xr-x  12 Angel  staff    384 Mar  2 13:53 charts
drwxr-xr-x  10 Angel  staff    320 Mar  2 13:53 files
drwxr-xr-x   8 Angel  staff    256 Mar  2 13:53 subcharts
drwxr-xr-x  61 Angel  staff   1952 Mar  2 13:53 templates
-rw-r--r--@  1 Angel  staff   7165 Feb 13 14:07 user-values.yaml.example
-rw-r--r--@  1 Angel  staff   4396 Feb 13 14:07 values-ocp.yaml
-rw-r--r--@  1 Angel  staff  33535 Feb 13 14:07 values.yaml


We will need to manipulate just one of them but, before that, let's generate some secrets:

 ### 3.1 Generate secrets 

We need to generate two internal passwords (secrets) for our deployment and we will ensure that the format will comply with the expectations. Additionally, we will retrieve other variables that we will use in the next steps. Run the following cell and take a look at the output values because you may need to copy-and-paste them later.

In [20]:
# Run this cell

# these are the secrets
export FERNET_KEY=$(dd if=/dev/urandom bs=32 count=1 2>/dev/null | openssl base64)
export WEBSERVER_KEY=$(head -c 32 /dev/urandom | base64 | tr -d =)

# these are other values we retrieved before but we get them once again to be sure that they are not lost
export DATABAND_PROJECT=databand 
export REGISTRY_USER=$(oc whoami)
export REGISTRY_PASS=$(oc whoami -t)
export REGISTRY_URL=$(oc get route -n openshift-image-registry | grep default-route | awk '{print $2}')

# this is simply to retrieve the version number (image tag)
export IMAGE_TAG=$(docker images | grep dbnd-webserver | head -1 | awk '{print $2}' | sed 's/python-//')

# this is just to verify that the variables are not empty. You may need these values later
echo DATABAND_PROJECT=$DATABAND_PROJECT
echo REGISTRY_USER=$REGISTRY_USER
echo REGISTRY_PASS=$REGISTRY_PASS
echo REGISTRY_URL=$REGISTRY_URL
echo IMAGE_TAG=$IMAGE_TAG
echo FERNET_KEY=$FERNET_KEY
echo WEBSERVER_KEY=$WEBSERVER_KEY

DATABAND_PROJECT=databand
REGISTRY_USER=IAM#angelito@de.ibm.com
REGISTRY_PASS=sha256~YUoJrPZBkAtjead6CDeSveXNplatuHwKqId5xXmqRVM
REGISTRY_URL=default-route-openshift-image-registry.itzroks-1100005cc8-iw6gvy-6ccd7f378ae819553d37d5f2ee142bd6-0000.us-east.containers.appdomain.cloud
IMAGE_TAG=v1.0.19.7
FERNET_KEY=uAggDbMoUmRhDsB5lKur72qcyt4GkDkV+e/u4mKxANU=
WEBSERVER_KEY=YCxSxY0zsFTaS74Lk7UoyL3PoNIxC5EzFjURr3YTUDE


### 3.2 Customize the deployment file

Now, we will change the file `user-values.yaml` to customize our deployment. You need to choose one of two options:

- a. manual edit
- b. automatic edit

It may be a good idea to start with the option a. but if you find it too complicated try b.

#### 3.2.a Manual edit of `user-values.yaml`

Check that you are in the right directory, i.e. containing the contents of the helm charts

In [57]:
cd $DATABAND_UNCOMPRESSED_DIR/databand
pwd
ls -l 

/Users/Angel/Downloads/databand-1.0.19-helm-chart/databand
total 184
-rw-r--r--@  1 Angel  staff   1098 Feb 13 14:07 Chart.lock
-rw-r--r--@  1 Angel  staff   1624 Feb 13 14:07 Chart.yaml
-rw-r--r--@  1 Angel  staff  11343 Feb 13 14:07 LICENSE
drwxr-xr-x   3 Angel  staff     96 Feb 27 12:05 LICENSES
-rw-r--r--@  1 Angel  staff  18137 Feb 13 14:07 README.md
drwxr-xr-x  12 Angel  staff    384 Feb 27 12:05 charts
drwxr-xr-x  10 Angel  staff    320 Feb 27 12:05 files
drwxr-xr-x   8 Angel  staff    256 Feb 27 12:05 subcharts
drwxr-xr-x  61 Angel  staff   1952 Feb 27 12:05 templates
-rw-r--r--@  1 Angel  staff   7165 Feb 13 14:07 user-values.yaml.example
-rw-r--r--@  1 Angel  staff   4396 Feb 13 14:07 values-ocp.yaml
-rw-r--r--@  1 Angel  staff  33535 Feb 13 14:07 values.yaml


Create the file `user-values.yaml` by copying  `user-values.yaml.example` 

In [60]:
cp user-values.yaml.example user-values.yaml
ls -l user-values*

-rw-r--r--@ 1 Angel  staff  7165 Feb 27 13:43 user-values.yaml
-rw-r--r--@ 1 Angel  staff  7165 Feb 13 14:07 user-values.yaml.example


Edit the file `user-values.yaml` end ensure that it looks like this (use the prover values for your environment that we retrieved before):

![](../pictures/user-values-yaml.png)

Save the file and you are done. Skip 3.2.b if you are fine with the result.

#### 3.2.b Automatic edit of user-values.yaml

If you don't want to edit the `user-values.yaml` file manually, the following cells will do the work for you.

First, you need to install a small utility that changes yaml files from the command line. It is called [`yq`](https://github.com/mikefarah/yq)

In [ ]:
# Install yq if you want to edit the user-values.yaml automatically
# use brew on MacOS
# alternatively, you may need to use apt, yum, snap or simply download the binary.
# go here for instructions https://github.com/mikefarah/yq

brew install yq

Ensure that you are in the directory where the helm charts were unpacked and create the file `user-values.yaml` from the example

In [21]:
# Run this cell to create a user-values.yaml file

cd $DATABAND_UNCOMPRESSED_DIR
cd databand
cp user-values.yaml.example user-values.yaml
pwd
ls -l user-values.yaml


/Users/Angel/Downloads/databand-1.0.19-helm-chart/databand
-rw-r--r--@ 1 Angel  staff  7165 Mar  2 13:54 user-values.yaml


This file contains some default values that we will need to modify. This is how they look like now:

In [22]:
# These commands display the values that we will edit automatically

echo repository=$(yq '.global.databand.image.repository' user-values.yaml) 
echo tag=$(yq '.global.databand.image.tag' user-values.yaml)
echo registry=$(yq '.global.databand.imageCredentials.registry' user-values.yaml)
echo username=$(yq '.global.databand.imageCredentials.username' user-values.yaml)
echo password=$(yq '.global.databand.imageCredentials.password' user-values.yaml)
echo fernetKey=$(yq '.databand.fernetKey' user-values.yaml)
echo webKey=$(yq '.web.secret_key' user-values.yaml)
echo datastage=$(yq '.dbnd-datastage-monitor.enabled' user-values.yaml)

repository=registry.gitlab.com/databand/databand-deploy
tag=v1.0.19.7
registry=registry.gitlab.com
username=
password=
fernetKey=qOUpd5Mjzzt6E6G9m7_EViAiWqXiN-GMrVvfBzOGluc=
webKey=
datastage=false


The following cell perfoms the actual edit. Now, you understand why we retrieved and exported some variables above.

In [23]:
# Change the file user-values.yaml 

yq -i '.global.databand.image.tag = strenv(IMAGE_TAG)' user-values.yaml
yq -i '.global.databand.imageCredentials.registry = strenv(REGISTRY_URL)' user-values.yaml
yq -i '.global.databand.imageCredentials.username = strenv(REGISTRY_USER)' user-values.yaml
yq -i '.global.databand.imageCredentials.password = strenv(REGISTRY_PASS)' user-values.yaml
export YAML_REGISTRY=$REGISTRY_URL/$DATABAND_PROJECT
yq -i '.global.databand.image.repository = strenv(YAML_REGISTRY)' user-values.yaml 
yq -i '.databand.fernetKey = strenv(FERNET_KEY)' user-values.yaml
yq -i '.web.secret_key = strenv(WEBSERVER_KEY)' user-values.yaml
yq -i '.dbnd-datastage-monitor.enabled = true ' user-values.yaml 

If you wish to review the changes, run the following cell and compare the values.

In [25]:
echo repository=$(yq '.global.databand.image.repository' user-values.yaml) 
echo tag=$(yq '.global.databand.image.tag' user-values.yaml)
echo registry=$(yq '.global.databand.imageCredentials.registry' user-values.yaml)
echo username=$(yq '.global.databand.imageCredentials.username' user-values.yaml)
echo password=$(yq '.global.databand.imageCredentials.password' user-values.yaml)
echo fernetKey=$(yq '.databand.fernetKey' user-values.yaml)
echo webKey=$(yq '.web.secret_key' user-values.yaml)
echo datastage=$(yq '.dbnd-datastage-monitor.enabled' user-values.yaml)

repository=default-route-openshift-image-registry.itzroks-1100005cc8-iw6gvy-6ccd7f378ae819553d37d5f2ee142bd6-0000.us-east.containers.appdomain.cloud/databand
tag=v1.0.19.7
registry=default-route-openshift-image-registry.itzroks-1100005cc8-iw6gvy-6ccd7f378ae819553d37d5f2ee142bd6-0000.us-east.containers.appdomain.cloud
username=IAM#angelito@de.ibm.com
password=sha256~YUoJrPZBkAtjead6CDeSveXNplatuHwKqId5xXmqRVM
fernetKey=uAggDbMoUmRhDsB5lKur72qcyt4GkDkV+e/u4mKxANU=
webKey=YCxSxY0zsFTaS74Lk7UoyL3PoNIxC5EzFjURr3YTUDE
datastage=true


Done! Optionally, you may want to display the file `user-values.yaml` and verify that is similar to the picure of the previous section

### 3.3 Run the helm deployment

The actual deployment of databand is done with a single command from the helm charts directory (the one containing the file that we've just edited)

In [26]:
cd $DATABAND_UNCOMPRESSED_DIR
cd databand
helm upgrade databand --install --namespace databand --values ./values-ocp.yaml --values ./user-values.yaml .

Release "databand" does not exist. Installing it now.
NAME: databand
LAST DEPLOYED: Thu Mar  2 13:56:54 2023
NAMESPACE: databand
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Congratulations. You have just deployed Databand

1. Access the Databand Web UI by running these commands:

   export SERVICE_NAME=$(kubectl get svc --namespace databand -l "component=web,app=databand" -o jsonpath="{.items[0].metadata.name}")
   kubectl port-forward --namespace databand svc/$SERVICE_NAME 8090:8090

   Databand Web UI is available in your web browser via http://127.0.0.1:8090


The output of the deplyoment command will display the way to access databand with port-forwarding, but there is another way. We just need to create the route that exposes the service.

In [29]:
oc project databand
oc get svc
oc expose svc databand-web

Already on project "databand" on server "https://c102-e.us-east.containers.cloud.ibm.com:31723".
NAME                               TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)    AGE
databand-alert-defs-syncer         ClusterIP   172.21.228.28    <none>        5000/TCP   61m
databand-celery-flower             ClusterIP   172.21.113.138   <none>        5555/TCP   61m
databand-dbnd-datastage-monitor    ClusterIP   172.21.27.143    <none>        8006/TCP   61m
databand-postgresql                ClusterIP   172.21.115.202   <none>        5432/TCP   61m
databand-postgresql-hl             ClusterIP   None             <none>        5432/TCP   61m
databand-prometheus-alertmanager   ClusterIP   172.21.212.49    <none>        9093/TCP   61m
databand-prometheus-server         ClusterIP   172.21.126.52    <none>        9090/TCP   61m
databand-redis-headless            ClusterIP   None             <none>        6379/TCP   61m
databand-redis-master              ClusterIP   172.21.184.247   <n

: 1

In the next section, you will see how to verify that databand is running

## 4. Test Databand

We have just created a databand instance with the default user `databand` and password `databand`. If you want to test the deployment for the first time , go to the OpenShift console and proceed as follows:
![](../pictures/opendataband.png)

Type the defaul credentials 

![](../pictures/logindataband.png)

And the main dashboard of databand will start. Note that it is empty after the deployment.

![](../pictures/maindataband.png)


---

Next Section: [Airflow deployment](./3_airflow_deploy.ipynb).   Previous Section: [Databand deployment](./2_databand_deploy.ipynb)

[Return to main](../README.md)
